# Try Azure ML Studio
This interactive Jupyter Notebook will allow you to setup infrastructure on Azure ML and authenticate with its nodes.

Make sure you have created a `SUBSCRIPTION_ID` [GitHub Codespace secret](https://docs.github.com/en/codespaces/managing-your-codespaces/managing-encrypted-secrets-for-your-codespaces) enabled the following GitHub Codespace secrets

Optionally, you can set the following Codespace secrets that allows to reuse existing Azure IDs and accounts:

``` SUBSCRIPTION_ID
RESOURCE_GROUP
WORKSPACE_NAME
WORKSPACE_REGION
```

These are retrieved and set from the next cell

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group = os.getenv("RESOURCE_GROUP", default="demo-try-azureml")
workspace_name = os.getenv("WORKSPACE_NAME", default="demo-try-azureml")
workspace_region = os.getenv("WORKSPACE_REGION", default="eastus2")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
    # write the details of the workspace to a configuration file to the notebook library
    ws.write_config()
    print("Workspace configuration succeeded.")
except Exception:
    print("Workspace not accessible. Ensure that Codespaces secrets are available, see first cell")
    raise

## Create the workspace 
Use the previously created configuration file to create the Azure ML workspace

In [ ]:
# Create the workspace using the specified parameters
ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,
                      create_resource_group = True,
                      exist_ok = True)
ws.get_details()

# write the details of the workspace to a configuration file to the notebook library
ws.write_config()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# define aml compute target(s) to create
amlcomputes = {
    "cpu-cluster": {
        "vm_size": "STANDARD_DS3_V2",
        "min_nodes": 0,
        "max_nodes": 3,
        "idle_seconds_before_scaledown": 240,
    }
}

# create aml compute targets
for ct_name in amlcomputes:
    if ct_name not in ws.compute_targets:
        compute_config = AmlCompute.provisioning_configuration(**amlcomputes[ct_name])
        ct = ComputeTarget.create(ws, ct_name, compute_config)
        ct.wait_for_completion(show_output=True)

## Perform exploratory work
You can now interact with the Azure ML Workspace, like train models, register datasets and models, or create an AutoML run

In [ ]:
# exploratory work

## Destroy workspace and resources
The next cell will destroy all assets from the Azure ML workspace

In [ ]:
ws.delete(delete_dependent_resources=True, no_wait=False)